In [15]:
include("slim_reader.jl")
using Knet

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at slim_reader.jl:6
└ @ Core /home/ec2-user/comp492/Text2Image-cemil-slim-dataset-reader/slim_reader.jl:6


In [61]:
BATCH_SIZE = 50
slim_data_dir = "/home/ec2-user/efs/slim-dataset"

64

In [17]:
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)

Dict{String,Any} with 4 entries:
  "sess"         => PyObject <tensorflow.python.client.session.Session object a…
  "filenames"    => PyObject <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=stri…
  "iterator"     => PyObject <tensorflow.python.data.ops.iterator_ops.Iterator …
  "next_element" => (((PyObject <tf.Tensor 'IteratorGetNext_2:0' shape=(?, 9, ?…

In [18]:
# Iterate through the dataset and get the next batch
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);

In [205]:
aimages, acaptions, acameras = getdata(syn_train_batch)

(K32(50,10,128,128,3)[0.6039216⋯], ["There is a purple capsule in front of a pink ico. The capsule is close to the ico.  " "There is a pink ico close to a purple capsule. The ico is to the right of the capsule.  " … "There is a purple capsule close to a pink ico. The capsule is in front of the ico.  " "There is a pink ico to the right of a purple capsule. The ico is close to the capsule.  "; "There is a red ico in front of a violet ico. There is a red ico in front of a large peach box. The ico is to the right of the box. There is a large peach box to the left of a violet ico.  " "There is a red ico in front of a violet ico. There is a red ico to the right of a large peach box. The ico is in front of the box. There is a violet ico to the right of a large peach box.  " … "There is a red ico to the left of a violet ico. There is a red ico to the left of a large peach box. The ico is in front of the box. There is a violet ico in front of a large peach box.  " "There is a red ico in front o

In [224]:
# create a dictionary of known words
vocab = Dict()
vocab = Dict("" => KnetArray(zeros(EMBEDDING_SIZE))) # both padding and whitespace

Dict{String,KnetArray{Float64,1}} with 1 entry:
  "" => K64(64)[0.0⋯]

In [225]:
caption_embeds = arrange(acaptions,vocab)

50-element Array{Any,1}:
 Any[K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯]]
 Any[K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯]]
 Any[K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,60)[0.03361509787334013⋯], K64(64,6

In [226]:
vocab

Dict{String,KnetArray{Float64,1}} with 47 entries:
  "pink"      => K64(64)[-0.05320388718541161⋯]
  "cone"      => K64(64)[-0.010810351692621983⋯]
  "right"     => K64(64)[0.028482869615203465⋯]
  "capsule"   => K64(64)[0.039905934207305094⋯]
  "teal"      => K64(64)[0.06035515198046429⋯]
  "of"        => K64(64)[0.16980724761717605⋯]
  "purple"    => K64(64)[0.1549221604706835⋯]
  "cylinder"  => K64(64)[-0.17402959506774507⋯]
  "right."    => K64(64)[-0.0033173125895600286⋯]
  "to"        => K64(64)[-0.027362710963867082⋯]
  "box."      => K64(64)[-0.0035722392186845753⋯]
  "ico"       => K64(64)[-0.04894584681424219⋯]
  "is"        => K64(64)[-0.05887074007516242⋯]
  "close"     => K64(64)[-0.006768370161991888⋯]
  "cylinder." => K64(64)[-0.006094151433673034⋯]
  "sphere."   => K64(64)[0.0946939455871608⋯]
  "sphere"    => K64(64)[0.08425054758431308⋯]
  "cone."     => K64(64)[0.15491235124517147⋯]
  "in"        => K64(64)[0.1387726196094268⋯]
  "There"     => K64(64)[0.033615097873

In [223]:
@doc conv4

```
conv4(w, x; kwargs...)
```

Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.

Currently KnetArray{Float32/64,4/5} and Array{Float32/64,4} are supported as `w` and `x`.  If `w` has dimensions `(W1,W2,...,I,O)` and `x` has dimensions `(X1,X2,...,I,N)`, the result `y` will have dimensions `(Y1,Y2,...,O,N)` where

```
Yi=1+floor((Xi+2*padding[i]-Wi)/stride[i])
```

Here `I` is the number of input channels, `O` is the number of output channels, `N` is the number of instances, and `Wi,Xi,Yi` are spatial dimensions.  `padding` and `stride` are keyword arguments that can be specified as a single number (in which case they apply to all dimensions), or an array/tuple with entries for each spatial dimension.

# Keywords

  * `padding=0`: the number of extra zeros implicitly concatenated at the start and at the end of each dimension.
  * `stride=1`: the number of elements to slide to reach the next filtering window.
  * `dilation=1`: dilation factor for each dimension.
  * `mode=0`: 0 for convolution and 1 for cross-correlation.
  * `alpha=1`: can be used to scale the result.
  * `handle`: handle to a previously created cuDNN context. Defaults to a Knet allocated handle.


In [258]:
filter = KnetArray(Knet.xavier(3))
filter = reshape(filter, (3,1,1,1))
input = KnetArray(vec(caption_embeds[1][1]))
input = reshape(input, (length(input), 1, 1, 1))

3840×1×1×1 KnetArray{Float64,4}:
[:, :, 1, 1] =
  0.03361509787334013 
 -0.1445966248805982  
  0.09551723425175648 
 -0.06614023148448502 
 -0.03198389239714411 
  0.003817265135760889
  0.01215813232125082 
  0.1289966705604129  
  0.09349869907719485 
 -0.16249734978917435 
 -0.044088484520822174
 -0.1245009887448101  
  0.06594251444029123 
  ⋮                   
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 

In [259]:
@show y = conv4(filter, input, dilation=2)

y = conv4(filter, input, dilation=2) = K64(3836,1,1,1)[-0.0013128866591483887⋯]


3836×1×1×1 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.0013128866591483887
  0.06546028331279256  
 -0.06078689139514584  
  0.07809300185397507  
  0.05011377705393124  
 -0.013634450398669925 
  0.006853039640640006 
 -0.1590872894994064   
 -0.046960879430028    
  0.020432394396476414 
  0.06642493002874204  
  0.009467297098555443 
 -0.028657186067584245 
  ⋮                    
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  

In [260]:
yy = conv4(filter, y, dilation=2)

3832×1×1×1 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.0018120180105365598
 -0.019242203604278172 
  0.051707561348136075 
 -0.09689274294323333  
 -0.04107333071463137  
 -0.03214648215609802  
  0.0023889227436859934
  0.10064955758175703  
  0.03777834857256113  
 -0.006358481677636829 
 -0.0648036199637567   
 -0.0062734059280039655
  0.00980464358551326  
  ⋮                    
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  

In [261]:
yyy = conv4(filter, yy, dilation=2)

3828×1×1×1 KnetArray{Float64,4}:
[:, :, 1, 1] =
  0.0037004402837051238
 -0.02667053574762711  
 -0.041061786326762384 
  0.07689273704130226  
  0.035773124956968584 
  0.045859540596445    
 -0.009903287189667225 
 -0.061857097528221244 
 -0.037673836719154655 
  0.002108775241411728 
  0.06100986727925491  
  0.00739996978613829  
  0.018019626407904773 
  ⋮                    
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  

In [299]:
captions_hat = vec(yyy)

3828-element KnetArray{Float64,1}:
  0.0037004402837051238
 -0.02667053574762711  
 -0.041061786326762384 
  0.07689273704130226  
  0.035773124956968584 
  0.045859540596445    
 -0.009903287189667225 
 -0.061857097528221244 
 -0.037673836719154655 
  0.002108775241411728 
  0.06100986727925491  
  0.00739996978613829  
  0.018019626407904773 
  ⋮                    
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  

In [283]:
function build_angles(tuple)
    sin_value = tuple[3]/sqrt(tuple[1]^2+tuple[3]^2)
    cos_value = tuple[1]/sqrt(tuple[1]^2+tuple[3]^2)
    return KnetArray([sin_value, cos_value])
end

build_angles (generic function with 1 method)

In [289]:
tuple = build_angles(acameras[1,1,:])

2-element KnetArray{Float32,1}:
  0.8953934 
 -0.44527593

In [286]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

In [287]:
struct Layer0; w; b; end
Layer0(i::Int,o::Int) = Layer0(param(o,i),param0(o))
(l::Layer0)(x) = (l.w * x .+ l.b)

In [292]:
mlp1=Chain(Layer0(2,32)) # MLP1 dimensionality 32

Chain((Layer0(P(KnetArray{Float32,2}(32,2)), P(KnetArray{Float32,1}(32))),))

In [293]:
mlp2=Chain(Layer0(3860,2500), Layer0(2500,1000), Layer0(1000,256)) # MLP2 dimensionality 256

Chain((Layer0(P(KnetArray{Float32,2}(2500,3860)), P(KnetArray{Float32,1}(2500))), Layer0(P(KnetArray{Float32,2}(1000,2500)), P(KnetArray{Float32,1}(1000))), Layer0(P(KnetArray{Float32,2}(256,1000)), P(KnetArray{Float32,1}(256)))))

In [295]:
cameras_hat = mlp1(tuple)

32-element KnetArray{Float32,1}:
  0.24294043 
 -0.21395203 
  0.044674255
 -0.3009952  
  0.02855713 
 -0.24835156 
  0.065606184
  0.109428644
 -0.11246404 
  0.0844585  
  0.06966905 
 -0.025629437
 -0.13189846 
  ⋮          
  0.04445757 
  0.23382267 
 -0.1637316  
 -0.27894253 
  0.2100029  
  0.18465818 
 -0.19198799 
  0.14908513 
 -0.22744024 
  0.11463731 
  0.17712262 
  0.07698359 

In [305]:
captions_hat = convert(KnetArray{Float32},captions_hat)

3828-element KnetArray{Float32,1}:
  0.0037004403
 -0.026670536 
 -0.041061785 
  0.07689273  
  0.035773125 
  0.04585954  
 -0.0099032875
 -0.061857097 
 -0.03767384  
  0.0021087753
  0.06100987  
  0.0073999697
  0.018019626 
  ⋮           
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         
  0.0         

In [308]:
mlp2(vcat(captions_hat, cameras_hat))

256-element KnetArray{Float32,1}:
  0.0009552671
  0.00499776  
 -0.009077348 
  0.0035285754
 -0.0039222194
  0.013600332 
  0.009764381 
  0.0028035284
 -0.00468146  
 -0.01868543  
  0.008617871 
  0.0011645677
 -0.0042066462
  ⋮           
  0.012515955 
 -0.005956823 
 -0.020680724 
 -0.009527057 
 -0.005908364 
 -0.0012455045
 -0.009382162 
  0.0040890644
 -0.017256409 
 -0.0013365897
  0.017339882 
  0.016975498 